## Let us convert our date example to a class based design

##### based on current feature set

In [58]:
class DateUtils:
    
    def is_leap_year(year):
        return year % (4 if year%100!=0 else 400) ==0

    def days_in_month(month, year=2001):
        if month==2:
            return 28 if not DateUtils.is_leap_year(year) else 29
        elif month<8 and month%2==1 or month>=8 and month%2==0:
            return 31
        else:
            return 30

    def day_value(dd,mm,yyyy):
       
        y=yyyy-1
        days =   y*365  \
            + y//4   \
            - y//100 \
            + y//400
        
        # to this add days in month till mm-1
        m=1
        while m< mm:
            days+= DateUtils.days_in_month(m,yyyy)
            m+=1

        # add dd to the running total
        return days+dd

    def week_day_number(dd,mm,yyyy):
        ref_date_value= DateUtils.day_value(1,1,2012)
        date_value = DateUtils.day_value(dd,mm,yyyy)

        diff = date_value - ref_date_value
        return diff%7

    def day_name(index):
        
        day_names=("Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday")
        return day_names[index]


    def week_day_name(dd,mm,yyyy):
        d=DateUtils.week_day_number(dd,mm,yyyy)
        return DateUtils.day_name(d)
    
    def calendar(mm,yyyy):
        print(f'Calendar for {mm}/{yyyy}')
        print('Sun\tMon\tTue\tWed\tThu\tFri\tSat')
        s= DateUtils.week_day_number(1,mm,yyyy)
        print("\t"*s,end="")    
        
        for dd in range(1,DateUtils.days_in_month(mm,yyyy)+1):
            print(dd,end="\t")
            s+=1
            if s%7==0:
                print()

        print()

    def month_name(month):
        months= ("","January","February","March","April","May","June","July","August","September","October","November","December")
        return months[month]
        

### Let's create a class to represent a single date

In [59]:
class Date:
    def __init__(self, dd,mm,yyyy):
        if mm<1 or yyyy<1 or dd<1 or mm>12 or dd>DateUtils.days_in_month(mm,yyyy):
            raise ValueError(f"Invalid date")        
        self.dd=dd
        self.mm=mm
        self.yyyy=yyyy

    def __str__(self):
        day=DateUtils.week_day_name(self.dd,self.mm,self.yyyy)
        month=DateUtils.month_name(self.mm)
        return f'{day}, {month} {self.dd}, {self.yyyy}'


        



In [60]:
d=Date(4,9,2023)
print(d)

Monday, September 4, 2023


### More converter Functions

* python provides several special functions to convert object from one type to anther.
* important functions include
    * \_\_str\_\_
        * to convert to string
    * \_\_int\_\_
        * to convert to int
    * \_\_bool\_\_
        * to convert to bool


In [61]:
class Date:
    def __init__(self, dd,mm,yyyy):
        if mm<1 or yyyy<1 or dd<1 or mm>12 or dd>DateUtils.days_in_month(mm,yyyy):
            raise ValueError(f"Invalid date")        
        self.dd=dd
        self.mm=mm
        self.yyyy=yyyy

    def __str__(self):
        day=DateUtils.week_day_name(self.dd,self.mm,self.yyyy)
        month=DateUtils.month_name(self.mm)
        return f'{day}, {month} {self.dd}, {self.yyyy}'

    def __int__(self):
        return DateUtils.day_value(self.dd,self.mm,self.yyyy)

        



In [62]:
d=Date(30,1,1948)
print(int(d))

711156


#### can I convert number 711156 to the date 30/01/1948?
* we should start counting days from 1/1/0001

In [68]:

def days_in_year(y):
    return 365 + (1 if DateUtils.is_leap_year(y) else 0)

def int_to_date( day_value):
    
    y=1
    while True:
        year_value=days_in_year(y)
        if day_value>year_value:
            y+=1
            day_value-= year_value
           # print(day_value, y,year_value)
        else:
            break;


    m=1
    while True:
        month_value=DateUtils.days_in_month(m,y)
        
        if day_value>month_value:
            day_value-=month_value
            m+=1
        else:
            
            break

    d=day_value
    
   
    return (d,m,y)



In [74]:
class Date:
    def __init__(self, dd, mm=None, yyyy=None):
        if mm == None and yyyy==None:
            x=int_to_date(dd)
            self.dd=x[0]
            self.mm=x[1]
            self.yyyy=x[2]
        elif yyyy>0 and mm>0 and mm<=12 and dd>=1 and dd<= DateUtils.days_in_month(mm,yyyy):
            self.dd=dd
            self.mm=mm
            self.yyyy=yyyy
        else:
            raise Exception(f"Invalid Date value {dd}/{mm}/{yyyy}")


    def __str__(self):
        day=DateUtils.week_day_name(self.dd,self.mm,self.yyyy)
        month=DateUtils.month_name(self.mm)
        return f'{day}, {month} {self.dd}, {self.yyyy}'

    def __int__(self):
        return DateUtils.day_value(self.dd,self.mm,self.yyyy)

    

In [76]:
d=Date(30,1,1948)
print(d)
v= int(d)
print(v)

Friday, January 30, 1948
711156


In [77]:
d2=Date(v)
print(d2)

Friday, January 30, 1948


### Python operator overloading

* python allows us to overload existing python operators (like +,-,*,/) to work for user defined objects like (Date)

* to overload an operator we have to write a python special method where LHS will be self and RHS can be any value 

* important operators in this section include
    * Arithmetic operators
        * \_\_add\_\_  -> x+y
        * \_\_sub\_\_  -> x-y
        * \_\_mul\_\_  -> x*y
        * \_\_realdiv\_\_  -> x/y
        * \_\_intdiv\_\_\ --> x//y
        * \_\_mod\_\_  ---> x%y

    * relational operators
        * \_\_eq\_\_ --> x==y
        * \_\_ne\_\_ --> x!=y
        * \_\_le\_\_  ---> x<y
        * ...

### can these operators help us in date class?

* we may implement
    * date + day ---> next date
        * what will be 10 days after "d"  --> d+10
    * date -day ---> previous date

    * date - date  ---> gap between two days

In [78]:
class Date:
    def __init__(self, dd, mm=None, yyyy=None):
        if mm == None and yyyy==None:
            x=int_to_date(dd)
            self.dd=x[0]
            self.mm=x[1]
            self.yyyy=x[2]
        elif yyyy>0 and mm>0 and mm<=12 and dd>=1 and dd<= DateUtils.days_in_month(mm,yyyy):
            self.dd=dd
            self.mm=mm
            self.yyyy=yyyy
        else:
            raise Exception(f"Invalid Date value {dd}/{mm}/{yyyy}")


    def __str__(self):
        day=DateUtils.week_day_name(self.dd,self.mm,self.yyyy)
        month=DateUtils.month_name(self.mm)
        return f'{day}, {month} {self.dd}, {self.yyyy}'

    def __int__(self):
        return DateUtils.day_value(self.dd,self.mm,self.yyyy)

    def __add__(self, days):
        x = int(self)+days
        return Date(x)



In [80]:
d=Date(4,9,2023)
print(d)

d2=d+110
print(d2)

Monday, September 4, 2023
Saturday, December 23, 2023


In [81]:
d1 = Date(30,1,1948)
d2 = Date(30,1,1948)
d3 = d2

In [82]:
d1==d2

False

In [83]:
d2==d3

True

### What happened

* Here we can have created only two dates
    * first is referred by d1 second is referred by d2

* d3 is not a new date. It is reference to same date as d2

* == by default checks for same reference (id)

* Ideally it should consider even d1 and d2 to be same as they all represent the same date



### Assignment 4.1

* implement following features
1. date1 - int = date2
    * what date was n days before date1
2. date1- date2  = int 
    * gap between the two date
2. date1==date2 should return true if the two represent same date 
    * (may be different references)

In [ ]:
### What happened?